### Imports and Setting Configs

In [244]:
import pandas as pd
import urllib.parse
from collections import defaultdict
from IPython.display import display, HTML
import log_analysis as LA

from importlib import reload

pd.set_option('display.max_columns', None)

# CSS = """
# .output {
#     flex-direction: row;
# }
# """

# HTML('<style>{}</style>'.format(CSS))

### Loading the Dataset
- 7 Days of Data from Sumologic: `_sourceCategory=prod_courservice_access "feedback.v1"`
- Sample rate of 0.1

In [187]:
df = pd.read_csv("feedback-logs.csv")
df = df.drop(['_collector', 'ipforwarded', 'parentspanid', '_source', '_sourcecategory', '_sourcename', 'spanid', 'timestamp', 'traceid', 'traceidhex', '_raw'], axis=1)
df.head()

,_messagetimems,_messagetime,code,duration,fromservicename,method,toplevelresource,actionname,courserasource,findername,ipsrc,requestid,responsebodylength,_size,_sourcehost,url,userid
0,1529088264428,06/15/2018 11:44:24.428 -0700,200,17,-,GET,feedback.v1,NaN,-,course,34.227.148.239,HvrEPnDMEeiVcQ7s3Ns7fg,1945,486,ip-10-1-26-108,www.coursera.org/api/feedback.v1?courseId=yOZE...,26216411
1,1529088256562,06/15/2018 11:44:16.562 -0700,403,5,-,GET,feedback.v1,NaN,-,course,54.227.153.99,GpqeqnDMEeiA1w7A_uReMA,123,482,ip-10-1-26-108,www.coursera.org/api/feedback.v1?courseId=3Nog...,22227164
2,1529088243164,06/15/2018 11:44:03.164 -0700,200,28,-,GET,feedback.v1,NaN,-,course,54.166.131.175,EqQyRnDMEeiD4xIPKc1ZHg,2454,457,ip-10-1-30-198,www.coursera.org/api/feedback.v1?courseId=1RBl...,16409906
3,1529088220712,06/15/2018 11:43:40.712 -0700,403,1,-,GET,feedback.v1,NaN,-,course,54.166.131.175,BVenp3DMEeiMpgotq0wepg,129,473,ip-10-1-1-188,www.coursera.org/api/feedback.v1?courseId=2iD7...,-
4,1529088219942,06/15/2018 11:43:39.942 -0700,403,5,-,GET,feedback.v1,NaN,-,course,18.206.95.50,BOG0enDMEei2chJ5pbpoVA,122,481,ip-10-1-27-164,www.coursera.org/api/feedback.v1?courseId=FXS0...,5162615


### Dataset Cleaning

#### Creating columns for each query

In [247]:
def get_request_query(request: str) -> dict:
    messy_queries = urllib.parse.parse_qs(urllib.parse.urlsplit(request).query)
    return {k : v[0] for k, v in messy_queries.items()}

def get_query_dataframe(urls):
    queries = urls.apply(get_request_query)
    
    # Create a set of keys throughout all queries
    unique_keys = set()
    for query in queries:
        unique_keys.update(query.keys())
    
    # Create a dict of columns. Use an empty string if key isn't in the query
    column_index = dict(enumerate(unique_keys))
    new_query_data = defaultdict(list)
    for query in queries:
        for i, column_name in column_index.items():
            if column_name in query:
                new_query_data[column_name].append(query[column_name])
            else:
                new_query_data[column_name].append(None)
    return pd.DataFrame(new_query_data)
    

In [248]:
query_df = get_query_dataframe(df.url)
query_df.head()

,includes,ratingValues,q,courseId,categories,courseCompleted,courseBranchId,feedbackSystem,itemId,start,limit,fields,page
0,onDemandCourseMaterials.v2(modules),"4,5",course,yOZEQ3lwEeWb-BLhFdaGww,generic,None,None,STAR,None,0,4,"onDemandCourseMaterials.v2(moduleIds,modules)",None
1,onDemandCourseMaterials.v2(modules),"4,5",course,3NogQ11pEeWJEg4Ftlh5hw,generic,None,None,STAR,None,0,4,"onDemandCourseMaterials.v2(moduleIds,modules)",None
2,users,"4,5",course,1RBlaf9AEea_bxJuN8v34g,generic,None,None,STAR,None,0,4,"users,userPublicProfiles.v1(fullName,photoUrl)",None
3,onDemandCourseMaterials.v2(modules),"4,5",course,2iD7dxkvEeW9dA4X94-nLQ,generic,None,None,STAR,None,0,4,"onDemandCourseMaterials.v2(moduleIds,modules)",None
4,onDemandCourseMaterials.v2(modules),"4,5",course,FXS00V1oEeWK1woL5P9cGQ,generic,None,None,STAR,None,0,4,"onDemandCourseMaterials.v2(moduleIds,modules)",None


#### Added columns to main dataframe

In [190]:
df = df.join(query_df,how='outer')

#### Dataset Size

In [191]:
len(df)

88756

#### Example URL

In [192]:
df.url[0]

'www.coursera.org/api/feedback.v1?courseId=yOZEQ3lwEeWb-BLhFdaGww&ratingValues=4%2C5&q=course&fields=onDemandCourseMaterials.v2%28moduleIds%2Cmodules%29&categories=generic&start=0&feedbackSystem=STAR&includes=onDemandCourseMaterials.v2%28modules%29&limit=4'

## Body Response Lengths
#### _A good proxy for unique responses_

Note that of the top 5 lengths by frequency, 4 of those lengths are all _403_ responses

In [193]:
# display_side_by_side("Frequency of Response Body Lengths:", "403 Response Body Lengths:")

print("Response Length for all requests\n", df.responsebodylength.value_counts()[:5], sep="")
print()
print("Response Length for 403 requests\n", df[df.code == 403].responsebodylength.value_counts()[:5], sep="")


Response Length for all requests
129    33290
123    15814
122     4089
74       688
121      592
Name: responsebodylength, dtype: int64

Response Length for 403 requests
129    33290
123    15814
122     4089
121      592
120       99
Name: responsebodylength, dtype: int64


### Latencies 👍

In [194]:
print("0.50 percentile:", df.duration.quantile(q=.5), "ms")
print("0.95 percentile:", df.duration.quantile(q=.95), "ms")
print("0.99 percentile:", df.duration.quantile(q=.9), "ms")

0.50 percentile: 7.0 ms
0.95 percentile: 252.0 ms
0.99 percentile: 63.0 ms


### HTTP Response Codes
_The large majority of responses are 403 Forbidden responses_

In [195]:
response_code_frequencies = df.code.value_counts()
print("Response Codes\n", response_code_frequencies, sep="")
print("\n")
print("Fraction of responses that are error codes:", response_code_frequencies[403] / response_code_frequencies.sum())

Response Codes
403    53898
200    34555
504      257
401       20
405       16
400        7
500        3
Name: code, dtype: int64


Fraction of responses that are error codes: 0.6072603542295733


## Parsing Data on The HTTP Request

### Most requests are coming from 25 "URLs". But most of them are 403s

In [196]:
df200 = df[df.code == 200]
df403 = df[df.code == 403]

In [197]:
request_frequencies = list(df.url.value_counts()[:15])
request_frequencies403 = list(df403.url.value_counts()[:15])
print("Top 15 Total Request Frequencies:", sum(request_frequencies))
print()
print("{0: <20}".format("Top 15 Requests:"), request_frequencies)
print("{0: <20}".format("Top 15 403 Requests:"), str(request_frequencies403))

Top 15 Total Request Frequencies: 42131

Top 15 Requests:     [5733, 5282, 4273, 3218, 2815, 2751, 2358, 2349, 2336, 2307, 2252, 1918, 1540, 1527, 1472]
Top 15 403 Requests: [5733, 5282, 4273, 3218, 2815, 2751, 2358, 2349, 2336, 2307, 2252, 1918, 1540, 1472, 1396]


## `feedback.v1` Queries

### `courseId` Quick look

### The most commonly requested courseIds are to the same group of classes. 
_There are only 64 courses for which people looked for reviews_

In [198]:
reload(LA)

<module 'log_analysis' from '/Users/dgrossman/python-notebooks/log_analysis.py'>

In [199]:
num_requests_by_course_id = df.courseId.value_counts()
print("{0: <25}".format("Number of unique course ids: "), len(num_requests_by_course_id200.keys()))

Number of unique course ids:  64


### `feedbackSystem` Quick look

In [200]:
num_requests_by_feedback_system = df.feedbackSystem.value_counts()
num_requests_by_feedback_system200 = df200.feedbackSystem.value_counts()
print("NUMBER OF REQESTS BY FEEDBACK SYSTEM - OVERALL")
print(num_requests_by_feedback_system, '\n\n')
print("NUMBER OF REQESTS BY FEEDBACK SYSTEM - 200")
print(num_requests_by_feedback_system200)

NUMBER OF REQESTS BY FEEDBACK SYSTEM - OVERALL
STAR                 88670
FLAG                    78
LIKE_OR_DISLIKE          3
* STAR                   2
* LIKE_OR_DISLIKE        2
Name: feedbackSystem, dtype: int64 


NUMBER OF REQESTS BY FEEDBACK SYSTEM - 200
STAR               34496
FLAG                  58
LIKE_OR_DISLIKE        1
Name: feedbackSystem, dtype: int64


... I think I was  the `LIKE_OR_DISLIKE` Query

### `categories` Quick look

In [201]:
num_requests_by_feedback_system = df.categories.value_counts()
num_requests_by_feedback_system200 = df200.categories.value_counts()
print("NUMBER OF REQESTS BY CATEGORIES - OVERALL")
print(num_requests_by_feedback_system, '\n\n')
print("NUMBER OF REQESTS BY CATEGORIES - 200")
print(num_requests_by_feedback_system200)

NUMBER OF REQESTS BY CATEGORIES - OVERALL
generic      88669
content         30
offensive       25
grading         13
subtitle         8
technical        1
audio            1
Name: categories, dtype: int64 


NUMBER OF REQESTS BY CATEGORIES - 200
generic      34496
content         23
offensive       18
grading         10
subtitle         5
technical        1
audio            1
Name: categories, dtype: int64


### `ratingValues` Quick look

In [202]:
num_requests_by_rating_values = LA.get_queries_by_key_counts(request_queries, 'ratingValues')
num_requests_by_rating_values200 = LA.get_queries_by_key_counts(request_queries200, 'ratingValues')
print("NUMBER OF REQESTS BY RATING VALUES - OVERALL")
print(num_requests_by_rating_values, '\n\n')
print("NUMBER OF REQESTS BY RATING VALUES - 200")
print(num_requests_by_rating_values200)

NUMBER OF REQESTS BY RATING VALUES - OVERALL
4,5                       87504
1,2,3,4,5                  1068
0                            78
5                            26
1                            23
3                            19
4                            15
2                            15
0,1,2,3,4,5,6,7,8,9,10        4
dtype: int64 


NUMBER OF REQESTS BY RATING VALUES - 200
4,5          33544
1,2,3,4,5      887
0               58
3               19
2               14
4               13
1               13
5                7
dtype: int64


In [204]:
df.head()

,_messagetimems,_messagetime,code,duration,fromservicename,method,toplevelresource,actionname,courserasource,findername,ipsrc,requestid,responsebodylength,_size,_sourcehost,url,userid,includes,ratingValues,q,courseId,categories,courseCompleted,courseBranchId,feedbackSystem,itemId,start,limit,fields,page
0,1529088264428,06/15/2018 11:44:24.428 -0700,200,17,-,GET,feedback.v1,NaN,-,course,34.227.148.239,HvrEPnDMEeiVcQ7s3Ns7fg,1945,486,ip-10-1-26-108,www.coursera.org/api/feedback.v1?courseId=yOZE...,26216411,onDemandCourseMaterials.v2(modules),"4,5",course,yOZEQ3lwEeWb-BLhFdaGww,generic,None,None,STAR,None,0,4,"onDemandCourseMaterials.v2(moduleIds,modules)",None
1,1529088256562,06/15/2018 11:44:16.562 -0700,403,5,-,GET,feedback.v1,NaN,-,course,54.227.153.99,GpqeqnDMEeiA1w7A_uReMA,123,482,ip-10-1-26-108,www.coursera.org/api/feedback.v1?courseId=3Nog...,22227164,onDemandCourseMaterials.v2(modules),"4,5",course,3NogQ11pEeWJEg4Ftlh5hw,generic,None,None,STAR,None,0,4,"onDemandCourseMaterials.v2(moduleIds,modules)",None
2,1529088243164,06/15/2018 11:44:03.164 -0700,200,28,-,GET,feedback.v1,NaN,-,course,54.166.131.175,EqQyRnDMEeiD4xIPKc1ZHg,2454,457,ip-10-1-30-198,www.coursera.org/api/feedback.v1?courseId=1RBl...,16409906,users,"4,5",course,1RBlaf9AEea_bxJuN8v34g,generic,None,None,STAR,None,0,4,"users,userPublicProfiles.v1(fullName,photoUrl)",None
3,1529088220712,06/15/2018 11:43:40.712 -0700,403,1,-,GET,feedback.v1,NaN,-,course,54.166.131.175,BVenp3DMEeiMpgotq0wepg,129,473,ip-10-1-1-188,www.coursera.org/api/feedback.v1?courseId=2iD7...,-,onDemandCourseMaterials.v2(modules),"4,5",course,2iD7dxkvEeW9dA4X94-nLQ,generic,None,None,STAR,None,0,4,"onDemandCourseMaterials.v2(moduleIds,modules)",None
4,1529088219942,06/15/2018 11:43:39.942 -0700,403,5,-,GET,feedback.v1,NaN,-,course,18.206.95.50,BOG0enDMEei2chJ5pbpoVA,122,481,ip-10-1-27-164,www.coursera.org/api/feedback.v1?courseId=FXS0...,5162615,onDemandCourseMaterials.v2(modules),"4,5",course,FXS00V1oEeWK1woL5P9cGQ,generic,None,None,STAR,None,0,4,"onDemandCourseMaterials.v2(moduleIds,modules)",None


In [241]:
df[(df.code == 403)].limit.value_counts()

4     53708
10      190
Name: limit, dtype: int64

In [271]:
print(df[df.code == 200].includes.value_counts())
print("\n")
print(df[df.code == 403].includes.value_counts())

users                                  29248
onDemandCourseMaterials.v2(modules)     4296
Name: includes, dtype: int64


onDemandCourseMaterials.v2(modules)    53617
users                                     91
Name: includes, dtype: int64
